<a href="https://colab.research.google.com/github/anubhavgupta1/Dive-Into-Deep-Learning/blob/main/Deep%20Learning%20Computation/Layers%20and%20Blocks/tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Deep Learning Computation

In [ ]:
!pip install d2l==0.16.1

In [2]:
import tensorflow as tf

In [3]:
X = tf.random.uniform((2, 20))

### A Custom Block

In [4]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
])

In [5]:
print(net(X))

tf.Tensor(
[[ 0.02454413  0.05374416 -0.22445574  0.10537914 -0.01967596  0.05305097
  -0.02177824  0.24214353  0.31233367  0.06279922]
 [-0.12228318  0.12440787 -0.04207617 -0.141327   -0.04125782  0.15450881
   0.2045113   0.192507    0.15914743  0.03142395]], shape=(2, 10), dtype=float32)


### MLP Block

In [6]:
class MLP(tf.keras.Model):
    # Declare a layer with model parameters. Here, we declare two fully
    # connected layers
    def __init__(self):
        # Call the constructor of the `MLP` parent class `Block` to perform
        # the necessary initialization. In this way, other function arguments
        # can also be specified during class instantiation, such as the model
        # parameters, `params` (to be described later)
        super().__init__()
        # Hidden layer
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)  # Output layer

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input `X`
    def call(self, X):
        return self.out(self.hidden((X)))

In [7]:
net = MLP()
print(net(X))

tf.Tensor(
[[ 0.12711485  0.16342229 -0.48333412  0.089953   -0.12214594  0.24096318
   0.18865812  0.1727748  -0.07359472 -0.02697731]
 [-0.09546676  0.08119087 -0.3592066   0.10021627  0.0293078   0.00357199
   0.00810516  0.37822014 -0.0345638   0.07874873]], shape=(2, 10), dtype=float32)


### The Sequential Block

In [8]:
class MySequential(tf.keras.Model):
    def __init__(self, *args):
        super().__init__()
        self.modules = []
        for block in args:
            # Here, `block` is an instance of a `tf.keras.layers.Layer`
            # subclass
            self.modules.append(block)

    def call(self, X):
        for module in self.modules:
            X = module(X)
        return X

In [9]:
net = MySequential(
    tf.keras.layers.Dense(units=256, activation=tf.nn.relu),
    tf.keras.layers.Dense(10))

In [10]:
print(net(X))

tf.Tensor(
[[ 0.00583003  0.01177476  0.12718418 -0.5266906  -0.01847959 -0.20649724
  -0.06827422  0.209494   -0.21486029  0.41507924]
 [ 0.22579806 -0.30515245  0.2524678  -0.35757127  0.30900776  0.01641573
   0.26814288  0.12637533 -0.3970175   0.4101131 ]], shape=(2, 10), dtype=float32)


### Ensemble Block

In [11]:
class FixedHiddenMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        # Random weight parameters created with `tf.constant` are not updated
        # during training (i.e., constant parameters)
        self.rand_weight = tf.constant(tf.random.uniform((20, 20)))
        self.dense = tf.keras.layers.Dense(20, activation=tf.nn.relu)

    def call(self, inputs):
        X = self.flatten(inputs)
        # Use the created constant parameters, as well as the `relu` and
        # `matmul` functions
        X = tf.nn.relu(tf.matmul(X, self.rand_weight) + 1)
        # Reuse the fully-connected layer. This is equivalent to sharing
        # parameters with two fully-connected layers
        X = self.dense(X)
        # Control flow
        while tf.reduce_sum(tf.math.abs(X)) > 1:
            X /= 2
        return tf.reduce_sum(X)

In [12]:
net = FixedHiddenMLP()
print(net(X))

tf.Tensor(0.5202366, shape=(), dtype=float32)


In [13]:
class NestMLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.net = tf.keras.Sequential()
        self.net.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
        self.net.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
        self.dense = tf.keras.layers.Dense(16, activation=tf.nn.relu)

    def call(self, inputs):
        return self.dense(self.net(inputs))

In [14]:
net = NestMLP()
print(net(X))

tf.Tensor(
[[0.09285448 0.49414733 0.         0.0551398  0.         0.
  0.         0.         0.29116362 0.         0.         0.
  0.6010559  0.33207092 0.19420975 0.09575619]
 [0.0252568  0.68659973 0.         0.         0.         0.
  0.         0.         0.40465826 0.15191303 0.21433485 0.
  0.3473198  0.02466469 0.1318965  0.27913833]], shape=(2, 16), dtype=float32)


In [15]:
chimera = tf.keras.Sequential()
chimera.add(NestMLP())
chimera.add(tf.keras.layers.Dense(20))
chimera.add(FixedHiddenMLP())

In [16]:
print(chimera(X))

tf.Tensor(0.7573608, shape=(), dtype=float32)
